<style>
        a { text-decoration: none;
            font-weight: normal;
            color: #0000aa;
          }
          
      img {
            width: 35%;
            height: auto;
          }

     code { font-weight: 600; }

     h1,h2,h3,h4,h5,h6 { font-weight: 500; }
        
</style>

# Data Wrangling: OpenStreetMaps Data Case Study
### Patrick Cook  03/11/2021

![Round Rock City Boundary](images/round_rock_map.JPG "Round Rock city boundary")

## Map Area
Round Rock, TX, United States
* https://www.openstreetmap.org/relation/115318

* https://overpass-api.de/api/map?bbox=-97.8181,30.4570,-97.5267,30.5957

The Overpass API link was used to download the 220 MB osm file for Round Rock. The map file was placed in the [data folder](data "Link to Folder") and renamed to round_rock.xml.

This area was chosen because I am a resident so I have some domain knowledge. I am interested in finding out how well Round Rock is represented in OpenStreetMaps and the accuracy of the data. Tourism is important to my city's revenue and OpenStreetMaps is used when creating boundaries to measure tourism visitation patterns. In addition, businesses benefit from additional free sources of information that may drive customers to them. Therefore, it is important that they are represented in the data and the information is accurate and up-to-date. 


## Exploration and Issues Discovered in the Round Rock Data
**Process**: During the exploration phase, a sample of the data [round_rock_sample](data "Link to folder") was created using the create_sample function in the [explore_raw_data module](modules "Link to folder") to test functions on and better understand the structure of the data. The [`count_tags()`](modules\explore_raw_data.py "Link to containing .py file") was used to get all tag types found in the complete [round_rock.xml](data "Link to folder") file including number of occurences.

**Findings**: The original data contains 4 header tags (osm, meta, note and bounds) containing file information such as download date, time and position. Then it is followed by node, way and relation parent elements. The parent elements contain data entry and location information as attributes. The parents have child elements member, nd and tag. The majority of the descriptive information is coded in the 'tag' child element attributes. The elements found and the count of each is given below. The majority of tags are node and nd containing position and data entry information. The tag elements contain the descriptive information that will be used in the analysis. Therefore, the focus will be on extracting and structuring this data.

### Element Tag Types
    
    {
      'bounds': 1,
      'member': 19047,
      'meta': 1,
      'nd': 1152020,
      'node': 1027359,
      'note': 1,
      'osm': 1,
      'relation': 357,
      'tag': 328734,
      'way': 112084
    }

 

## Exploration of Attributes
To get an idea of the type, number and structure of tag's key attributes, [`categorize_tag_key_characters()`](modules\explore_raw_data.py "Link to containing .py file") was run. It was found that there are 248 unique tag keys that are all lower case, 189 tag keys that have a colon and are lower and 69 unique tag keys that contain capital letters or contain multiple colons. There are no tag keys with special problematic characters. The summary results are below.

``` python
{'problemchars': 0, 'lower': 226175, 'other': 1295, 'lower_colon': 101264}
```

```
There are:
          248 unique keys in lower,
          189 unique keys in lower_colon,
          0 unique keys in problemchars and
          69 unique keys in other.
```

## Auditing the Data
Reviewing the sample file data,  errors and non-standard data were found in many of the attribute key categories. I decided to focus on addr:street (street names), addr:postcode (postal codes) and phone (phone numbers) for this project. The street name and phone numbers are a focus as businesses need this information to be accurate for customers to contact them. The postal code was picked due to some outliers being found that are not postcodes of the city area.

### Auditing Adress Street Names  
**Process**: The audit street name function from the Udacity Data Wrangling course was modified to catch street names with special characters. The returned dictionary information was reviewed and used to modify the mapping dictionary and to update the expected_list of street type endings. The [`audit_street_validate_corrections()`](modules\audit_streets.py "Link to containing .py file") was modified to use the mapping dictionary to test corrections to the street names. 

**Findings**: The errors found during the street auditing process were:
* Streets with missing or non-standard street type endings.
    * Street names ending with Cv, Cv. and Cove
    * Street names ending with "Suite 301" or "United States"
* Non-standard formating for highways and directions.
    * IH-35, IH35, Interstate 35, I-35, Highway 35
    * North, N, N. and directions appearing at end of street name after street type 
* Steet names containing partial or full addresses such as
    * house number or apartment number
    * complete address written out with postal code, state and country

<br>
The following code are examples of street names found during auditing.


```   {
       ',': set(['11066 Pecan Park St  300, Cedar Park, TX',
                 '1335 E Whitestone Blvd T100, Cedar Park, TX 78613, United States',
                 'Louis Henna Blvd, TX 45 Frontage Road',
                 'N Interstate Hwy 35, Round Rock, TX 78681',
                 'S Interstate 35, #260']),
       '35': set(['Highway Interstate 35',
                  'N Interstate Hwy 35',
                  'North IH 35',
                  'North Interstate 35',
                  'S Interstate 35',
                  'S Interstate Highway 35',
                  'South Interstate 35']),
       685': set(['FM 685', 'Farm to Market 685']),
 'Barrhead': set(['Barrhead']),
       'Cv': set(['Copper Point Cv',
                  'Quiet Meadows Cv',
                  'Ripley Castle Cv',
                  'Secluded Willow Cv']),
      }            
```
To clean the street names, all corrections were added to one mapping dictionary so that specific, single instances needing cleaning would occur in the same step as the general programmatic cleaning of the street endings. This **caused a problem with partial cleanning of the street name** instead of the whole street name being cleaned. As an example, the street names "N. IH35" and "IH35" are both street names found in the data. The mapping contains these "N. IH35" : "North I-35" and also "IH35" : "I-35" to catch these instances. Since dictionaries are not ordered in Python 2.7, the N. IH35 was being caught by the "IH35" mapping key and converted to N. I-35 (partially cleaned) instead of "North I-35".

To fix the issue, a separate function [`dictionary_key_length_ordered_descending()`](modules\helper_functions.py "Link to containing .py file") was used to create an OrderedDict by key value length. This ensures the longer, more specific keys are found before the less specific keys. Separating out the mapping dictionaries and sending the data to both dictionaries would be another option that might reduce time and spacial complexity but would require further testing. Testing the function gave the expected results but did highlight issues with street names with valid endings but beginning with abreviation letters for directions (N, S, E, W). The cleaning function added a directions_mapping to check for this case. Examples of the cleaning function results during auditing are given below. 

### Sample Street Name Cleaning Results:
```
11066 Pecan Park Ste 300, Cedar Park, TX => Pecan Park Boulevard
S Bell Blvd., Suite 301 => South Bell Boulevard
U.S. 183 => US 183
1500 S. IH35 => South I-35
N. IH35, => North I-35
W. Parmer Lane => West Parmer Lane
N. IH 35 Pflugerville => North I-35
MCNEIL RD => McNeil Road
University Blvd => University Boulevard
Exchange Blvd => Exchange Boulevard
N Heatherwilde Blvd => N Heatherwilde Boulevard
E Palm Valley Blvd => E Palm Valley Boulevard
South Bell Blvd => South Bell Boulevard
200 University Blvd => University Boulevard
```

## Auditing Postal Codes
The postal codes were audited next using the [`audit_postacode()`](modules\audit_postcodes.py "Link to containing .py file") function. Post codes in the United States are comprised of 5 digits with the first digit giving the National Area, the next two digits giving a Sectional (Regional) Area and the last two digits giving the Delivery Area. Therefore all postcodes should be 5 digits. To validate the data, postcodes for the City of Round Rock were placed in a list to compare postcode values to. The results of the audit showed that all postal codes for the city were found in the data file. There were also 10 postal codes that were not listed as belonging to Round Rock. The results of the audit is shown below.

```
There was at least one instance of each Round Rock postcode found in the data.
The following postcodes are not Round Rock postcodes and need review.
    set(['78613', '78758', '78680', '78682', '78621', '78729', '787664', '78641', '78728-1275', '78750'])
Completed
```


### Validating the postcode data
Using [Round Rock's City GIS software](https://maps.roundrocktexas.gov/cityview/),  researching and exploring specific elements, the following information was discovered.
* The postcodes 78613, 78758, 78729, 78641 and 78750 are neighborhoods at the border of the city
* The postcode 78621 does not seem to be touching the border of Round Rock and will require further investigation.
* The postcodes 78680 and 78682 are listed as specially designated zoning regions used for PO Box, high volume or historical significance postcodes.
* The postcode 787664 is a typo and should be 78664. This will be verified using the elements street, longitude and latitude data.
* The postcode 78728-1275 has the expanded code added (ZIP+4). The expanded code will be removed during cleaning.

Running the [`explore_postcode_details()`](modules\audit_postcodes.py "Links to containing .py file") function on the two zip codes showed both street addresses are in Round Rock and will be corrected using a mapping dictionary during cleaning.
* The 78621 postcode (Hoody's) should be 78681 and is likely a typo (2 instead of 8 on keypad).
* The 787664 postcode (Self Storage Facility) is a typo and should be 78664.

## Audit Phone Numbers
Auditing the phone numbers using the [`audit_phones()`](modules\audit_phones.py "Link to containing .py file") function shows mostly conisitent data with variations between using dashes, spaces and parenthesis to delimenate the numbers. Other issues that were discovered are:
* Numbers using more that one deliminator such as spaces and parenthesis
* Numbers using no deliminators
* Unicode character code /2100 found intead of '-'
* Four numbers where found more than once in the data

The results of the phone audit are below.

```
All phone numbers were checked.

There are 45 malformed numbers, 4 duplicate numbers and 248 wellformed numbers.

The malformed numbers found are: 
```

```python
{'(512) 246-7941'       : 'fix_number',
 '+1 (512) 469-7000'    : 'fix_number',
 '+1 (512) 759-5900'    : 'fix_number',
 '+1 512 218 5062'      : 'fix_number',
 '+1 512 218 9888'      : 'fix_number',
 '+1 512 238 0820'      : 'fix_number',
 '+1 512 244 3737'      : 'fix_number',
 '+1 512 248 7000'      : 'fix_number',
 '+1 512 252 1133'      : 'fix_number',
 '+1 512 255 7000'      : 'fix_number',
 '+1 512 255 7530'      : 'fix_number',
 '+1 512 258 8114'      : 'fix_number',
 '+1 512 277 6959'      : 'fix_number',
 '+1 512 310 7600'      : 'fix_number',
 '+1 512 310 7678'      : 'fix_number',
 '+1 512 324 4000'      : 'fix_number',
 '+1 512 341 1000'      : 'fix_number',
 '+1 512 362 9525'      : 'fix_number',
 '+1 512 402 7811'      : 'fix_number',
 '+1 512 528 7000'      : 'fix_number',
 '+1 512 532 2200'      : 'fix_number',
 '+1 512 600 0145'      : 'fix_number',
 '+1 512 637 6890'      : 'fix_number',
 '+1 512 733 9660'      : 'fix_number',
 '+1 512 990 5413'      : 'fix_number',
 '+1 512)351 3179'      : 'fix_number',
 '+1 512-244-8500'      : 'fix_number',
 '+1 512-260-5443'      : 'fix_number',
 '+1 512-260-6363'      : 'fix_number',
 '+1 512-310-8952'      : 'fix_number',
 '+1 512-338-8805'      : 'fix_number',
 '+1 512-341-7387'      : 'fix_number',
 '+1 512-421-5911'      : 'fix_number',
 '+1 512-535-5160'      : 'fix_number',
 '+1 512-535-6317'      : 'fix_number',
 '+1 512-733-6767'      : 'fix_number',
 '+1 512-851-8777'      : 'fix_number',
 '+1 737 757 3100'      : 'fix_number',
 '+1512-413-9671'       : 'fix_number',
 '+1512-909-2528'       : 'fix_number',
 '+15123885728'         : 'fix_number',
 '+15124282300'         : 'fix_number',
 '+15124648382'         : 'fix_number',
 '1+512-696-5209'       : 'fix_number'}

There were 4 duplicate items found: 

set(['+1-512-428-2500', '+1-512-336-1328', '+1-512-310-8791', '+1-512-238-0475'])
```

### Cleaning the Phone Numbers
To make the data more consistent, it was decided to follow the US phone pattern with leading country code separated by dashes ( +1-###-###-#### ). The numbers that did not follow the US phone pattern were added as a key to the malformed dictionary. A partial clean was first performed using the [`phone_partial_clean()`](modules\helper_functions.py "Link to conataining .py file") function. This function programatically cleans the data by using the `replace()` string method calls to replace:

* All blanks with hyphens,
* Removes all parenthesis

Of the 45 original phone number corrections, the new dictionary still contained 8 phone numbers with errors (see below). These could be programatically cleaned using specific algorithms for each. The time complexity would increase significantly for so few corrections needed. Therefore, it was decided to manually edit these numbers in the mapping dictionary.

Numbers not fixed by partial clean (manually fixed):

```
 '(512) 246-7941'       : '512-246-7941',
 '+1 512)351 3179'      : '+1-512351-3179',
 u'+1-737-484\u20110700': u'+1-737-484\u20110700',
 '+1512-413-9671'       : '+1512-413-9671',
 '+1512-909-2528'       : '+1512-909-2528',
 '+15123885728'         : '+15123885728',
 '+15124282300'         : '+15124282300',
 '+15124648382'         : '+15124648382',
 
```

## Creating CSV Files and Database Import
With all the planned cleaning verified, the data was converted to csv files using the [`xml_to_csv()`](data.py "Link to containing .py file.") function in the data.py file. To identify attributes to be cleaned, the Tag element attributes are checked to see if they are in the update_list. If the attribute key is one of the attributes to be cleaned, they are sent to the [`update_values()`](modules\update_values.py "Link to containing .py file.") function which acts as flow control and sends the value to the appropriate cleaning function returning the cleaned value.

``` python
...
UPDATE_LIST = ['addr:street', 'addr:postcode', 'phone']
...
    for child in element.iter('tag'):
        tag_dict = {}

        if child.attrib['k'] in UPDATE_LIST:                                        
            child.attrib['v'] = update_value(child.attrib['k'], child.attrib['v'])
...
```

During the conversion to csv, the **entire dataset was validated using the cerberus package** and the [schema.py](schema.py "Link to .py file") file. Additionally, after the conversion to csv, the csv files were descriminately audited using filters to confirm corrections.

**Note**: Due to the time complexity of running the cerberus validation, smaller test.xml and round_rock_sample.xml files were run prior to validating the entire data set.

## SQL Import
After converting to csv files, the data was then loaded into an Sqlite3 database, [Round_RockDb.db](sql "Link opens containing folder"), and the function [`process_sql()`](modules\process_sql.py "Link Opens containing .py file") was used to create the tables for the database. Included in the [`process_sql()`](modules\process_sql.py "Link Opens containing .py file") function, is the table schema used to create the tables. The table schema is also available in the sql folder in the [create_table_schema](sql\create_table_schema.py "Link opens containing file") file for convenience.  

Finally, the csv files were loaded into the tables using the following code and the [`csv_to_sql()`](modules\process_sql.py "Link opens containing .py file.") function.
<br>
<br>


``` python
    # import csv to sqlite db tables: nodes_tags, nodes, ways_nodes, ways_tags, ways

    from modules.process_sql import csv_to_sql

    # Dictionary of file paths and table names
    csv_file_table ={'sql\\csv\\nodes_tags.csv' : 'nodes_tags', 
                     'sql\\csv\\nodes.csv' : 'nodes', 
                     'sql\\csv\\ways_nodes.csv' : 'ways_nodes', 
                     'sql\\csv\\ways_tags.csv' : 'ways_tags', 
                     'sql\\csv\\ways.csv' : 'ways'
                    }

    db_file = 'sql\\Round_RockDb.db'

    for csv_file, db_table in csv_file_table.items():
        csv_to_sql(csv_file, db_file, db_table)
```

### Database Queries: Statistical Overview of Dataset
The following sections contain information about the Size of the files compared to the database, number of unique users, number of nodes and ways, and number of:
* unique street names
* unique postcodes and 
* unique phone numbers

In addition, I will look at the occurrence counts in the data and the top 10 in the street names and postcode groups to find which are most represented in the data. I will also examine the phone numbers to determine which numbers appeared more than once in the data. 

### Size of Files
The [`get_file_info()`](modules\helper_functions.py "Link to containing .py file") helper function was used to gather information about specific files used. The original data was 220 MB in size and the database has reduces this size by 101 MB to 119 MB. The total of all the csv files are 135 MB.


```
    round_rock.xml...............220 MB
    round_rockdb.db..............119 MB
    nodes_tags.csv.................1 MB
    nodes.csv.....................92 MB
    ways_nodes.csv................26 MB
    ways_tags.csv..................9 MB
    ways.csv.......................7 MB
```

### Number of Unique Users

``` sql
    SELECT COUNT(DISTINCT (wnUnion.uid)) as "Total Unique Users" 
    FROM (SELECT uid FROM ways UNION ALL SELECT uid FROM nodes) as wnUnion;
```

Total Unique Users: **1054**

### Top 10 Users Represented in Data
The top 10 users are listed below. The data shows at least one user likely using two user ids and accounting for the top 2 appearances. The suffix "*_atxbuildings*" was found to be from an [openstreetmap guide](https://wiki.openstreetmap.org/wiki/Austin,_TX/Buildings_Import/Software_Setup) on an Austin, TX wiki page.
<br>

``` sql
    SELECT user, uid, COUNT(wnUnion.uid) as "Appearance Count" 
                   FROM (SELECT user, uid FROM ways 
                         UNION ALL 
                         SELECT user, uid FROM nodes) as wnUnion
                   GROUP BY user, uid
                   ORDER BY "Appearance Count" DESC
                   LIMIT 10;
```

<br>

|           user             |   |     uid     |   |  Appearance Count  |
|        ---------:          | - |  ---------: | - | -----------------: |
|  ccjjmartin__atxbuildings  |   |  3405475    |   |      312213        |
|   ccjjmartin_atxbuildings  |   |  3370181    |   |      309281        |
|    patisilva_atxbuildings  |   |  3369502    |   |      122028        |
|            SathyaPendyala  |   |  3618405    |   |       30931        |
|                     s0707  |   | 11358207    |   |       25687        |
|           woodpeck_fixbot  |   |   147510    |   |       24508        |
|       wilsaj_atxbuildings  |   |  3341346    |   |       24321        |
|                technogeek  |   |    98830    |   |       18930        |
|                    torapa  |   |  5446055    |   |       11552        |
|                   JanineG  |   | 12179240    |   |       11016        |

<br>
<br>
<br>
The "**_atxbuildings** reference was found in 5 users including the duplicated user cjmartin. Also, checking for other user names with "cjmartin" showed no other users with that name in their user name.  
<br>


``` sql
    SELECT user, uid, COUNT(wnUnion.uid) as "Appearance Count" 
                   FROM (SELECT user, uid FROM ways 
                         UNION ALL 
                         SELECT user, uid FROM nodes) as wnUnion
                   WHERE user LIKE '%atxbuildings%'
                   GROUP BY user, uid
                   ORDER BY "Appearance Count" DESC;
```

<br>

|           user             |   |     uid     |   |  Appearance Count  |
|        ---------:          | - |  ---------: | - | -----------------: |
|  ccjjmartin__atxbuildings  |   |  3405475    |   |      312213        |
|   ccjjmartin_atxbuildings  |   |  3370181    |   |      309281        |
|    patisilva_atxbuildings  |   |  3369502    |   |      122028        |
|       wilsaj_atxbuildings  |   |  3341346    |   |       24321        |
|  lyzidiamond_atxbuildings  |   |  3409435    |   |        2292        |

<br>

### Number of Nodes and Ways

The **nodes** table has **1027359 rows** and the **ways** table has **112084 rows**. 

```
    SELECT count(*) as "Nodes Count" FROM nodes;
    SELECT count(*) as "Ways Count" FROM ways;
```

### Number of Streets, Postcodes and Phone Numbers  
The results of streets, postcodes and phones in the data are:

|     keys  |      |   Count   |
|    ----:  | ---  |  -------: |
|   street  |      |   28533   |
| postcode  |      |    7110   |
|    phone  |      |     308   |

<br>
<br>

``` sql
SELECT key, COUNT(wnUnion.value) as "Appearance Count" 
                    FROM (SELECT key, value, type FROM ways_tags 
                          UNION ALL 
                          SELECT key, value, type FROM nodes_tags) as wnUnion
                    WHERE (type = 'addr' AND key IN ('street', 'postcode')) OR
                                 (type = 'regular' AND key = 'phone')
                    GROUP BY key
                    ORDER BY "Appearance Count" DESC;
```

### Top 10 Streets by Occurence in the Data
The top 10 streets by occurence are:  

**value**|    |**Unique Street Count**
:----- | --- |:-----:
Winding Shore Lane|   |142
East Whitestone Boulevard|   |135
Pencil Cactus Drive|   |131
Dashwood Creek Drive|   |127
Brushy Creek Road|   |117
Derby Day Avenue|   |116
Dorman Drive|   |110
Loch Linnhe Loop|   |109
Tortoise Street|   |106
Farm Pond Lane|   |104


<br>
<br>

```  sql
    SELECT value, COUNT(wnUnion.value) as "Unique Street Count" 
                        FROM (SELECT key, value, type FROM ways_tags 
                            UNION ALL 
                            SELECT key, value, type FROM nodes_tags) as wnUnion
                        WHERE key = 'street' AND type = 'addr'
                        GROUP BY value
                        ORDER BY "Unique Street Count" DESC
                        Limit 10;

```
<br>
<br>

Of the top 10 streets only **East Whitestone Blvd**, **Brushy Creek Road**, **Dorman Drive** and **Lock Linnhe Loop** are considered to be in the actual Round Rock city limits. The remaining 7 are in the map selection bounding box but are not officially in Round Rock's city limits. All streets except East Whitestone Blvd and Brushy Creek Road are residential roads. Based on this preliminary data, Round Rock businesses do not seem to be well represented in the data.

### Top 10 Postcodes Represented in Data

The following are the results of the top 10 postcodes by occurence. The bolded are either on the border or have a small area in the Round Rock city limits. The results show that a majority of the postcode data represents the Round Rock area. Further future study will be needed to find out if they are businesses or residential.

**Postcode**|   |**Count**
:-----:| --- |:-----:
78660|  |  1839
**78613**|  |  **1744**
78717|  |  755
78664|  |  558
78681|  |  460
78728|  |  453
**78729**|  |  **367**
78634|  |  293
**78641**|  |  **221**
78665|  |  201

<br>
<br>

```
    SELECT value, COUNT(wnUnion.value) as "Unique Postcode Count" 
                        FROM (SELECT key, value, type FROM ways_tags 
                            UNION ALL 
                            SELECT key, value, type FROM nodes_tags) as wnUnion
                        WHERE key = 'postcode' AND type = 'addr'
                        GROUP BY value
                        ORDER BY "Unique Postcode Count" DESC
                        Limit 10;
```

### Duplicate Phone Numbers
There were only 4 duplicated phone numbers in the data. The results are below. One of the duplicated numbers is for a middle school, one for a business wholesaler and the remaining two are for two CVS Pharmacy stores and two for Walgreens Pharmacy stores. This further leads to the conclusion that businesses may not be well represented in the data. Next I will check how many addresses contain I-35, SH 45, Louis Henna Boulevard and 620, which are all streets with primarily businesses.
<br>
<br>

**Phone Number** |     |**Count**
:-----           | --- |:-----:
+1-512-428-2500  |     |  2
+1-512-336-1328  |     |  2
+1-512-310-8791  |     |  2
+1-512-238-0475  |     |  2
+1-956-648-8580  |     |  1
+1-866-874-2389  |     |  1
+1-866-583-7952  |     |  1

<br>
<br>

``` sql
    SELECT value, COUNT(wnUnion.value) as "Unique Phone Count" 
                        FROM (SELECT key, value, type FROM ways_tags 
                            UNION ALL 
                            SELECT key, value, type FROM nodes_tags) as wnUnion
                        WHERE key = 'phone' AND type = 'regular'
                        GROUP BY value
                        ORDER BY "Unique Phone Count" DESC
                        Limit 10;
```

### Addresses Containing I-35, SH 45, Louis Henna or 620
The occurrences of street names with mostly business on them only show 248 records. Even more noticable from the query results below are that SH 45 shows only two business. This road is lined with businesses and should be much higher. The totals for I-35 and Ranch Road/Farm to Market 620 look more promising. There are likely several hundreds of businesses on each of these two roads but the totals for these two roads are better than SH 45. Louis Henna Boulevard's businesses are under represented. Knowing this area, there are likely over 100 businesses on this road.
<br>
<br>

**Street** |       |**Occurances**
 -----:     | :-----:| -----:
Ranch Road 620|  |82
North I-35 |  |62
South I-35 |  |29
I-35 |  |22
Louis Henna Boulevard|   |21
West Louis Henna Boulevard|  |17
North Ranch Road 620|  |9
North Farm to Market 620| |4
SH 45| |2  
**Total**| |**248**
<br>
<br>


``` sql
    SELECT value, COUNT(wnUnion.value) as "Occurance Count"
                        FROM (SELECT key, value, type FROM ways_tags 
                            UNION ALL 
                            SELECT key, value, type FROM nodes_tags) as wnUnion
                        WHERE (key = 'street' AND type = 'addr') AND
                            (value LIKE '%I-35%' OR value LIKE '%SH 45%' OR value LIKE '%620%' or value LIKE '%Louis Henna%')
                        GROUP BY value
                        ORDER BY "Occurance Count" DESC
                        Limit 20;
```

## Conclusion
The purpose of this project was to begin exploring, auditing and cleaning the OpenStreetMaps data for Round Rock, Texas. The node and way tag attributes for streets, postcodes and phone numbers were chosen to focus on how well businesses are represented in the data. It was found that the data set does contain valid data for many businesses but is not close to a complete representation of the total businesses in the city of Round Rock. The data was found to have a high level of consistency and uniformity with the postcodes, phone numbers and street names. Though, there were instances of inconsistency, with all three attribute values they did not represent a majority of the values. There were 28,533 street entries, 7,110 postcode entries and 308 phone entries. The streets had a little over 100 corrections, the postcodes had less than 10, and the phone number corrections were less than 30. This supports that most of the data is consistent and uniform. Most important to me is the data allowed me to deepen my knowledge and skills at data wrangling by working with real world data and solving real world problems. As always, data sets can always be improved. Some ideas of ways to improve the data are given in the next section.

### Additional Ideas for Improving the Data Including Benefits and Anticipated Problems
Though most of the data is consistent and uniform, data values that should follow a strict pattern are not universally uniform. As an example, in the data there were postcodes for the US that were 6 digits or had the zip+4 format. These could be easily checked using the regex pattern `^\d{5}$` before allowing them to be placed in the database. A similar pattern such as, `^\+1-[2-9]\d{2}-\d{3}-\d{4}$`, could be used to ensure consistency with phone numbers. If a pattern doesn't match, a user could be notified immediately and given information on formatting. Some anticipated problems with this improvement would be each country would need to have their own format pattern to check for and some mechanism would need to be in place to deal with any outliers that occur. An additional problem would be that being more strict tends to make some contributors stop contributing. Hopefully, the benefit of more consistent data would cause more businesses and developers of applications to those businesses to use the dataset. Therefore, canceling out the effect of the lost contributors. 

An additional thought is to make detailed standards for the naming conventions of common data values which can be programmatically checked similar to the audits done in this project. Place the standards prominently in an easily found place like the front-page menu bar. As an example of the challenge of finding a standard, while conducting a search for naming conventions I came across some OpenStreetMaps [Editing Standards and Conventions](https://wiki.openstreetmap.org/wiki/Editing_Standards_and_Conventions) through an external site but was unable to find other information on conventions used for phone numbers. When I finally found the conventions used for phone numbers, I found **two different methods** recommended as a standard. It is better to be consistent and not offer multiple ways of representing the same data. Only offer multiple choices when the data values are distinctly different such as with cuisines. Even then a menu of choices is better for the consistency and uniformity of the data. The query below gave 53 distinct cuisines found in the Round Rock data set. Some of the different representations, from the query, for the same items are:

* italian_pizza, pizza
* steak, steak_house
* wings, Wings
* american, American, local, regional
* ...

By offering a menu of items or at least a standard for naming items, these variations could be reduced making the data of a higher quality.

```sql
SELECT DISTINCT value
        FROM (SELECT key, value, type FROM ways_tags 
                UNION ALL 
                SELECT key, value, type FROM nodes_tags) as wnUnion
        WHERE (key = 'cuisine')
        ORDER BY value;
```

### References
https://mungingdata.com/sqlite/create-database-load-csv-python/

https://www.sqlitetutorial.net/sqlite-python/creating-database/

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

https://stackoverflow.com/questions/51463449/replace-csv-header-without-deleting-the-other-rows/51463964

https://stackoverflow.com/questions/35486721/how-to-prevent-use-of-the-first-row-pandas-dataframe-as-column-names-when-using

https://stackoverflow.com/questions/32213066/sqlite3-you-must-not-use-8-bit-bytestrings-unless-you-use-a-text-factory

https://stackoverflow.com/questions/12817151/how-to-get-column-names-with-query-data-in-sqlite3

https://maps.roundrocktexas.gov/cityview/

https://marketbusinessnews.com/financial-glossary/zip-code/

https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

https://thispointer.com/python-get-file-size-in-kb-mb-or-gb-human-readable-format/

https://wiki.openstreetmap.org/wiki/Austin,_TX/Buildings_Import/Software_Setup

https://wiki.openstreetmap.org/wiki/Editing_Standards_and_Conventions

https://wiki.openstreetmap.org/wiki/Key:phone
